In [2]:
import pandas as pd
import tensorflow as tf
import tempfile
import os
import numpy as np
from pathlib import Path

In [3]:
 from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/3w/msxr0yxx71d4flsgsqvzh5c80000gn/T/ipykernel_26867/1287427792.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [117]:
# define RGCN layer architecture for predictive cluster energy and structure model, set up forward and backward computation, n_layers = 11 (10 computational, 1 readout of SCF contraction coefficients) by default to ensure interdependency of electron density in all bonds on largest computed delocalized surfces
class RGCN_layer:
    def __init__(self, input_tensor, convolution_operation, computational_weight_paramaters, computational_bias_paramaters, computational_base_weights, computational_base_biases, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, orbital_overlap_schemes, activation_fxns, n_contraction_coefficients, atom_node_connections):
        self.input_tensor = input_tensor
        self.convolution_operation = convolution_operation
        self.weight_paramaters = weight_paramaters
        self.bias_paramaters = bias_paramaters
        self.base_weights = base_weights
        self.base_biases = base_biases
        self.orbital_overlap_schemes = orbital_overlap_schemes
        self.activation_fxn = activation_fxn
        self.n_contraction_coefficients = n_contraction_coefficients
        self.atom_node_connections = atom_node_connections


    def apply_convolution(self, convolution_operation, input_tensor):
        H_i = input_tensor
        H_i_T = H_i.transpose()
        H_i_conv_T = H_i_T @ convolution_operation
        H_i_conv = H_i_conv_T.transpose()
        return H_i_conv, H_i_conv_T

    def compute_weigths_given_orbital_overlap_types(self, orbital_overlap_schemes, base_weights, weight_paramaters, input_tensor):
        i = 0
        ith_state_weights = np.zeros((len(input_tensor), len(input_tensor)))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, computational_weight_paramaters will be set to an input_tensor * input_tensor matrix of ones, but will be set to the output of the update_weight_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_weights = weight_paramaters.transpose() @ base_weights
            ith_state_weights = ith_state_weights + overlap_type_weights
            i += 1
        A_RBT = sympy.symbols("A_RBT", real = True)
        b = sympy.symbols("b", real = True)
        weight_expression = sympy.concrete.summations.Sum(A_RBT * computational_base_weights, (b, 1, len(orbital_overlap_schemes)))
        return ith_state_weights, weight_expression

    def compute_biases_given_orbital_overlap_types(self, orbital_overlap_schemes, base_biases, bias_paramaters, input_tensor):
        i = 0
        ith_state_biases = np.zeros((len(input_tensor),))
        for i in np.arange(0, len(orbital_overlap_schemes)):
            # initially, computational_bias_paramaters will be set to an input_tensor * 1 vector of ones, but will be set to the output of the update_bias_paramaters function in the backpropagation for all paramater_update_count > 1
            overlap_type_biases = bias_paramaters @ base_biases
            ith_state_biases = ith_state_biases + overlap_type_biases
            i += 1
        A_RBB = sympy.symbols("A_RBB", real = True)
        b = sympy.symbols("b", real = True)
        bias_expression = sympy.concrete.summations.Sum(A_RBB * base_biases, (b, l, len(orbital_overlap_schemes)))
        return ith_state_biases, bias_expression


    def lin_reg_operation(self, convolution_operation, input_tensor, weight_paramaters, bias_paramaters, base_weights, base_biases, orbital_overlap_schemes, atom_node_connections):
        ith_state_weights, weight_expression = self.compute_weights_given_orbital_overlap_types(orbital_overlap_schemes, base_weights, weight_paramaters, input_tensor)
        ith_state_biases, bias_expression = self.compute_biases_given_orbital_overlap_typers(orbital_overlap_schemes, base_biases, bias_paramaters, input_tensor)
        H_i_conv, H_i_conv_T = self.apply_convolution(convolution_operation, input_tensor)
        H_i_lin = np.array([])
        H_i_lin = H_i_lin[np.newaxis, :]
        H_i_lin_reg = H_i_conv_T @ ith_state_weights + ith_state_biases
        i = 0
        for i in np.arange(0, len(atom_node_connections)):
            H_i_lin = np.append(H_i_lin, H_i_lin_reg[i] * (1/atom_node_connections[i]))
            i += 1
        H_i_lin = H_i_lin.reshape(H_i_lin, np.shape(H_i_lin_reg))
        H_i_lin_T = H_i_lin.transpose()
        CiR, W_R, H_i_conv_T, B_R = sympy.symbols("CiR", "W_R", "H_i_conv_T", "B_R", real = True)
        i = sympy.symbols("i", real = True)
        lin_reg_expression = sympy.concrete.summations.Sum((1/CiR) * (W_R * H_i_conv_T + B_R), (i, 1, len(atom_node_connections)))
        return H_i_lin, H_i_lin_T, lin_reg_expression

    
    def gelu(self, input_tensor):
        activation_output = (0.5 * input_tensor) * (1 + np.tanh(np.sqrt(2/np.pi) * (input_tensor + 0.044715*input_tensor**3)))
        return activation_output

    def forward_computational_graph_update(self, convolution_operation, input_tensor, computational_weight_paramaters, computational_bias_paramaters, computational_base_weights, computational_base_biases, orbital_overlap_schemes, activation_fxns, atom_node_connections):
        computational_activation = activation_fxns["computational"]
        H_i_conv, H_i_conv_T = self.apply_convolution(convolution_operation, input_tensor)
        ith_state_weights, ith_state_weight_expression = self.compute_computational_weights_given_orbital_overlap_types(orbital_overlap_schemes, computational_base_weights, computational_weight_paramaters, input_tensor)
        ith_state_biases, ith_state_bias_expression = self.compute_computational_biases_given_orbital_overlap_typers(orbital_overlap_schemes, computational_base_biases, computational_bias_paramaters, input_tensor)
        H_i_lin, H_i_lin_T, computational_lin_reg = self.computational_lin_reg_operation(convolution_operation, input_tensor, computational_weight_paramaters, computational_bias_paramaters, computational_base_weigts, computational_base_biases, orbital_overlap_schemes, atom_node_connections)
        if computational_activation == "gelu":
            H_i = self.gelu(H_i_lin)
            H_i = sympy.symbols("H_i", real = True)
            gelu_expression = (0.5 * H_i) * (1 + sympy.functions.elementary.hyperbolic.tanh((sympy.sqrt(2/np.pi)) * (H_i + 0.044715*H_i**3)))
        else:
            computational_activation = None
            pass
        return H_i, H_i_conv, H_i_conv_T, H_i_lin, H_i_lin_T, ith_state_weights, ith_state_biases, ith_state_weight_expression, ith_state_bias_expression, gelu_expression

    def forward_readout_graph_update(self, input_tensor, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, computational_weight_paramaters, computational_bias_paramaters, computational_base_weights, computational_base_biases, orbital_overlap_schemes, activation_fxns, n_contraction_coefficients, atom_node_connections):
        readout_activation = activation_fxns["readout"]
        nth_state_weights = self.compute_readout_weights_given_orbital_overlap_types(orbital_overlap_schemes, readout_base_weights, readout_weight_paramaters, input_tensor, n_contraction_coefficients)
        previous_layer_weights, nth_state_weight_expression = self.compute_computational_weigths_given_orbital_overlap_types(orbital_overlap_schemes, computational_base_weights, computational_weight_paramaters, input_tensor)
        nth_state_biases = self.compute_readout_biases_given_orbital_overlap_typers(orbital_overlap_schemes, readout_base_biases, readout_bias_paramaters, input_tensor, n_contraction_coefficients)
        previous_layer_biases, nth_state_bias_expression = self.compute_computational_biases_given_orbital_overlap_types(orbital_overlap_schemes, computational_base_biases, computational_bias_paramaters, input_tensor)
        H_n_lin, readout_lin_reg = self.readout_lin_reg_operation(input_tensor, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, orbital_overlap_schemes, n_contraction_coefficients, atom_node_connections)
        if readout_activation == "Lrelu":
            H_n = self.Lrelu(H_n_lin)
            H_n = sympy.symbols("H_n", real = True)
            Lrelu_expression = sympy.Piecewise((0.01 * H_n, H_n < 0), (H_n, H_n >= 0))
        else:
            readout_activation = None
            pass
        return H_n, H_n_lin, nth_state_weights, nth_state_biases, nth_state_weight_expression, nth_state_bias_expression, Lrelu_expression
            
        

            

In [118]:
, H_i_lin_T.transpose()[i] * (1/atom_node_connections[i]))
            i += 1
        H_i_lin = H_i_lin.reshape(H_i_lin, np.shape(H_i_lin_T.transpose()))
        H_i_lin_T = H_i_lin.transpose()
        CiR, W_R, H_i_conv_T, B_R = sympy.symbols("CiR", "W_R", "H_i_conv_T", "B_R", real = True)
        i = sympy.symbols("i", real = True)
        computational_lin_reg_expression = sympy.concrete.summations.Sum((1/CiR) * (W_R * H_i_conv_T + B_R), (i, 1, len(atom_node_connections)))
        return H_i_lin, H_i_lin_T, computational_lin_reg_expression

     def readout_lin_reg_operation(self, input_tensor, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, orbital_overlap_schemes, n_contraction_coefficients, atom_node_connections):
        nth_state_weights = self.compute_readout_weights_given_orbital_overlap_types(orbital_overlap_schemes, readout_base_weights, readout_weight_paramaters, input_tensor, n_contraction_coefficients)
        nth_state_biases = self.compute_readout_biases_given_orbital_overlap_typers(orbital_overlap_schemes, readout_base_biases, readout_bias_paramaters, input_tensor, n_contraction_coefficients)
        H_n_lin = np.array([])
        H_n_lin = H_n_lin[np.newaxis, :]
        H_n_lin_reg = input_tensor @ nth_state_weights + nth_state_biases
        i = 0
        for i in np.arange(0, len(atom_node_connections)):
            H_n_lin = np.append(H_n_lin, 
        H_n_lin = H_n_lin @ (1 / atom_node_connections)
        return H_n_lin

    def Lrelu(self, input_tensor):
        activation_output = np.maximum(input_tensor, 0.01 * input_tensor)
        return activation_output

    def gelu(self, input_tensor):
        activation_output = (0.5 * input_tensor) * (1 + np.tanh(np.sqrt(2/np.pi) * (input_tensor + 0.044715*input_tensor**3)))
        return activation_output


    def forward_readout_graph_update(self, input_tensor, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, orbital_overlap_schemes, activation_fxns, n_contraction_coefficients):
        readout_activation = activation_fxns["readout"]
        nth_state_weights = self.compute_readout_weights_given_orbital_overlap_types(orbital_overlap_schemes, readout_base_weights, readout_weight_paramaters, input_tensor, n_contraction_coefficients)
        nth_state_biases = self.compute_readout_biases_given_orbital_overlap_typers(orbital_overlap_schemes, readout_base_biases, readout_bias_paramaters, input_tensor, n_contraction_coefficients)
        H_n_lin = self.readout_lin_reg_operation(input_tensor, readout_weight_paramaters, readout_bias_paramaters, readout_base_weights, readout_base_biases, orbital_overlap_schemes, n_contraction_coefficients)
        if readout_activation == "Lrelu":
            H_n = self.Lrelu(H_n_lin)
            H_n = sympy.symbols("H_n", real = True)
            Lrelu_expression = sympy.Piecewise((0.01 * H_n, H_n < 0), (H_n, H_n >= 0))
        else:
            readout_activation = None
            pass
        return H_n, H_n_lin, nth_state_weights, nth_state_biases, Lrelu_expression
            
        

    

[[0. 1. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0.]]


In [6]:
mat_1 = np.array([[-1,3], [2,0]])
mat_2 = np.array([[7,-3], [-2,6]])
eigenval_1, eigenvec_1, = np.linalg.eig(mat_1)
eigenval_2, eigenvec_2 = np.linalg.eig(mat_2)
new_mat = np.array([[(-3/4), 1/2], [1/2, 0]])
new_mat_inv = np.linalg.inv(new_mat)
eigval, eigvec = np.linalg.eig(new_mat)
eigval_inv, eigvec_inv = np.linalg.eig(new_mat_inv)
print(eigval, eigval_inv)

[-1.    0.25] [-1.  4.]
